In [ ]:
import pandas as pd

# Loading the dataset
file_path = 'olist_order_reviews_dataset.csv'
df = pd.read_csv(file_path)

# Viewing the first lines and information
print(df.head())
print(df.info())

# Checking for null values
print(df.isnull().sum())

                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4                  NaN   
1             5                  NaN   
2             5                  NaN   
3             5                  NaN   
4             5                  NaN   

                              review_comment_message review_creation_date  \
0                                                NaN  2018-01-18 00:00:00   
1                                                NaN  2018-03-10 00:00:00   
2                                                NaN  2018-02-17 00:00:00   
3           

In [ ]:
# Removing lines where the comment is null
df_reviews = df.dropna(subset=['review_comment_message'])

# Creating the sentiment labels
# Grades 1 and 2 -> Negative (0)
# Grade 3 -> Neutral (let's ignore it for now to simplify the problem)
# Grades 4 and 5 -> Positive (1)
def score_to_sentiment(score):
    if score in [1, 2]:
        return 0  # Negative
    elif score in [4, 5]:
        return 1  # Positive
    else:
        return None # Neutral/Undefined

df_reviews['sentiment'] = df_reviews['review_score'].apply(score_to_sentiment)

# Removing neutral reviews (grade 3)
df_reviews = df_reviews.dropna(subset=['sentiment'])

# Selecting only the columns that interest us
df_final = df_reviews[['review_comment_message', 'sentiment']].copy()
df_final.rename(columns={'review_comment_message': 'text', 'sentiment': 'label'}, inplace=True)

# Converting the type of the 'label' column to integer
df_final['label'] = df_final['label'].astype(int)

# The dataset is large
# For a first test, let's take a balanced sample
# This makes training much faster
sample_size = 5000 # 5000 positives e 5000 negatives
df_sampled = df_final.groupby('label').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)

print(df_sampled['label'].value_counts())
print(df_sampled.head())

label
0    5000
1    5000
Name: count, dtype: int64
                                                text  label
0                                           n chegou      0
1  Um dos relogios veio aberto sem tampa e sem fu...      0
2  O produto chegou em 2 entregas, só que bem mal...      0
3  Ao invés de vocês se preocuparem em resolver o...      0
4  Estou aguardando entrega meu produto desde o d...      0


C:\Users\leors\AppData\Local\Temp\ipykernel_3200\3558854417.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['sentiment'] = df_reviews['review_score'].apply(score_to_sentiment)
C:\Users\leors\AppData\Local\Temp\ipykernel_3200\3558854417.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_final.groupby('label').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Loading the dataset from the Pandas DataFrame
dataset = Dataset.from_pandas(df_sampled)

# Loading the BERT tokenizer in Portuguese
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to tokenize texts
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Applying tokenization to the entire dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Splitting into training and testing (80% training, 20% testing)
final_datasets = tokenized_datasets.train_test_split(test_size=0.2)

print(final_datasets)

c:\codes\bert-sentiment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map:   0%|          | 0/10000 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 10000/10000 [00:00<00:00, 24685.84 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Loading the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Function to calculate metrics during training
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

# Creating the Data Collator, which will take care of the padding in each batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Defining the training arguments
training_args = TrainingArguments(
    output_dir="./results",         # Directory to save the model
    num_train_epochs=3,             # Number of training epochs
    per_device_train_batch_size=16, # Training batch size
    per_device_eval_batch_size=16,  # Evaluation batch size
    warmup_steps=500,               # Optimizer warm-up steps
    weight_decay=0.01,              # Weight decay
    logging_dir="./logs",           # Directory for logs
    logging_steps=100,
    evaluation_strategy="epoch",    # Evaluate at the end of each season
    save_strategy="epoch",          # Save the model at the end of each epoch
    load_best_model_at_end=True,    # Load the best model at the end
)

# Creating the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_datasets["train"],
    eval_dataset=final_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Begin training (this may take a while)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\codes\bert-sentiment-analysis\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/1500 [00:00<?, ?it/s]c:\codes\bert-sentiment-analysis\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  3%|▎         | 47/1500 [04:15<1:54:45,  4.74s/it]

KeyboardInterrupt: 

In [ ]:
# Evaluating the final model on the test set
evaluation_results = trainer.evaluate()
print("Resultados da Avaliação Final:")
print(evaluation_results)

# To make a prediction in a new text
from transformers import pipeline

# Create a text classification pipeline with the trained model
# trainer.save_model() saves the model to './results/checkpoint-...'
# Choose the checkpoint with the best result
best_model_path = "./results/checkpoint-XXXX" # Replace XXXX with the best checkpoint number
sentiment_analyzer = pipeline("sentiment-analysis", model=best_model_path, tokenizer=model_name)

# Usage example
texto_positivo = "Adorei o produto, chegou muito rápido e é de ótima qualidade!"
resultado_pos = sentiment_analyzer(texto_positivo)
print(f"Texto: '{texto_positivo}'\nResultado: {resultado_pos}")

texto_negativo = "Péssima experiência, o produto veio quebrado e a entrega demorou semanas."
resultado_neg = sentiment_analyzer(texto_negativo)
print(f"Texto: '{texto_negativo}'\nResultado: {resultado_neg}")